In [7]:
import functools
import time

import keras
import keras.backend as K
import matplotlib.pyplot as plt
# import scikitplot as skplt
import numpy as np
import pandas as pd
import scikitplot as skplt
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
from keras.layers import Dense, Dropout, Input, Lambda
from keras.models import Model, Sequential, load_model
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, accuracy_score, auc, auc, average_precision_score, average_precision_score, \
    confusion_matrix, confusion_matrix, pairwise_distances, precision_score, precision_score, recall_score, \
    recall_score, roc_auc_score, roc_auc_score, roc_curve, roc_curve
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from tqdm import tqdm

from data_analysis import calculate_metrics, get_rdkit_features
import dill

In [3]:
def get_keras_simple_nn(num_units=35, activation='relu', drop_rate=0.2):
    model = Sequential()
    model.add(Dense(num_units, activation=activation))
    model.add(Dropout(drop_rate))
    model.add(Dense(num_units, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
baselines_map = {
    'knn_clf': KNeighborsClassifier(n_neighbors=1, weights='distance'),
    'random_forest': RandomForestClassifier(),
    'logistic_regression': LogisticRegression(),
    'svc': SVC(gamma='auto'),
    'xgboost':
        xgb.XGBClassifier(
            objective='binary:logistic',
            booster='gbtree',
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=12,
            n_estimators=100,
            subsample=0.95
        ),
    'simple-NN': get_keras_simple_nn()
}

In [5]:
def get_baselines_performance(df_train, df_val, label_col='Binary', use_only=None):
    df_train = get_rdkit_features(df_train)
    df_val = get_rdkit_features(df_val)
    input_cols = [
        'BalabanJ', 'BertzCT', 'MaxAbsPartialCharge', 'MolLogP', 'MolWt', 'NumAliphaticCarbocycles',
        'NumRotatableBonds', 'RingCount', 'SlogP_VSA10', 'TPSA'
    ]
    if use_only is None:
        use_only = baselines_map.keys()
    metrics = {}
    for name, model in baselines_map.items():
        if name in use_only:
            if name == 'simple-NN':
                model.fit(df_train[input_cols].values, df_train[label_col].values, epochs=30, batch_size=32)
            else:
                model.fit(df_train[input_cols].values, df_train[label_col].values)

            y_pred = model.predict(df_val[input_cols].values).squeeze()
            y_true = df_val[label_col].values.squeeze()
            metrics[name] = calculate_metrics(y_true, y_pred)

    return pd.DataFrame(metrics).T

In [8]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)

In [9]:
training_p38 = [df_p38.loc[train_val_folds_p38[0][0]],
                 df_p38.loc[train_val_folds_p38[1][0]],
                 df_p38.loc[train_val_folds_p38[2][0]],
                 df_p38.loc[train_val_folds_p38[3][0]],
                 df_p38.loc[train_val_folds_p38[4][0]],
                 df_p38.loc[train_val_folds_p38[5][0]],
                 df_p38.loc[train_test_folds_p38[0]]
                 ]
validation_p38 = [df_p38.loc[train_val_folds_p38[0][1]],
                   df_p38.loc[train_val_folds_p38[1][1]],
                   df_p38.loc[train_val_folds_p38[2][1]],
                   df_p38.loc[train_val_folds_p38[3][1]],
                   df_p38.loc[train_val_folds_p38[4][1]],
                   df_p38.loc[train_val_folds_p38[5][1]],
                   df_p38.loc[train_test_folds_p38[1]]
                   ]

training_akt1 = [df_akt1.loc[train_val_folds_akt1[0][0]],
                 df_akt1.loc[train_val_folds_akt1[1][0]],
                 df_akt1.loc[train_val_folds_akt1[2][0]],
                 df_akt1.loc[train_val_folds_akt1[3][0]],
                 df_akt1.loc[train_val_folds_akt1[4][0]],
                 df_akt1.loc[train_val_folds_akt1[5][0]],
                 df_akt1.loc[train_test_folds_akt1[0]]
                 ]
validation_akt1 = [df_akt1.loc[train_val_folds_akt1[0][1]],
                   df_akt1.loc[train_val_folds_akt1[1][1]],
                   df_akt1.loc[train_val_folds_akt1[2][1]],
                   df_akt1.loc[train_val_folds_akt1[3][1]],
                   df_akt1.loc[train_val_folds_akt1[4][1]],
                   df_akt1.loc[train_val_folds_akt1[5][1]],
                   df_akt1.loc[train_test_folds_akt1[1]]
                   ]

training_pi3k = [df_pi3k.loc[train_val_folds_pi3k[0][0]],
                 df_pi3k.loc[train_val_folds_pi3k[1][0]],
                 df_pi3k.loc[train_val_folds_pi3k[2][0]],
                 df_pi3k.loc[train_val_folds_pi3k[3][0]],
                 df_pi3k.loc[train_val_folds_pi3k[4][0]],
                 df_pi3k.loc[train_val_folds_pi3k[5][0]],
                 df_pi3k.loc[train_test_folds_pi3k[0]]
                 ]
validation_pi3k = [df_pi3k.loc[train_val_folds_pi3k[0][1]],
                   df_pi3k.loc[train_val_folds_pi3k[1][1]],
                   df_pi3k.loc[train_val_folds_pi3k[2][1]],
                   df_pi3k.loc[train_val_folds_pi3k[3][1]],
                   df_pi3k.loc[train_val_folds_pi3k[4][1]],
                   df_pi3k.loc[train_val_folds_pi3k[5][1]],
                   df_pi3k.loc[train_test_folds_pi3k[1]]
                   ]

In [11]:
metrics_all = list()
for i in range(len(training_p38)):
    metrics_all.append(get_baselines_performance(training_p38[i],validation_p38[i]))

100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 318483.34it/s]

100%|███████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 87856.02it/s]

100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 212318.00it/s]

100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 636890.55it/s]

 17%|█████████████▎                                                               | 438/2541 [00:00<00:01, 1368.45it/s]


100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 255187.75it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 85059.20it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 170111.61it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 7290.90it/s]

100%|███████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 1850000.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 159.40it/s]
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:


Epoch 1/30
2541/2541 [==============================] - 0s 164us/step - loss: 7.9227 - acc: 0.5085
Epoch 2/30
2541/2541 [==============================] - 0s 156us/step - loss: 7.9163 - acc: 0.5089
Epoch 3/30
2541/2541 [==============================] - 0s 140us/step - loss: 7.9227 - acc: 0.5085
Epoch 4/30
2541/2541 [==============================] - 0s 141us/step - loss: 7.9225 - acc: 0.5081
Epoch 5/30
2541/2541 [==============================] - 0s 135us/step - loss: 7.9276 - acc: 0.5077
Epoch 6/30
2541/2541 [==============================] - 0s 131us/step - loss: 7.9227 - acc: 0.5085
Epoch 7/30
2541/2541 [==============================] - 0s 131us/step - loss: 7.9227 - acc: 0.5085
Epoch 8/30
2541/2541 [==============================] - 0s 136us/step - loss: 7.9227 - acc: 0.5085
Epoch 9/30
2541/2541 [==============================] - 0s 138us/step - loss: 7.9227 - acc: 0.5085
Epoch 10/30
2541/2541 [==============================] - 0s 133us/step - loss: 7.9227 - acc: 0.5085
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 318492.86it/s]

100%|███████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 87854.58it/s]

100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 231594.05it/s]

100%|██████████████████████████████████████████████████████████████████████████| 2541/2541 [00:00<00:00, 509622.08it/s]

  0%|                                                                                         | 0/2541 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 255187.75it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 85062.58it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 170125.17it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 6943.01it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 510253.52it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 169.21it/s]


...

Epoch 1/30
2545/2545 [==============================] - 0s 148us/step - loss: 7.7456 - acc: 0.5194
Epoch 2/30
2545/2545 [==============================] - 0s 130us/step - loss: 7.7456 - acc: 0.5194
Epoch 3/30
2545/2545 [==============================] - 0s 129us/step - loss: 7.7456 - acc: 0.5194
Epoch 4/30
2545/2545 [==============================] - 0s 130us/step - loss: 7.7456 - acc: 0.5194
Epoch 5/30
2545/2545 [==============================] - 0s 132us/step - loss: 7.7456 - acc: 0.5194
Epoch 6/30
2545/2545 [==============================] - 0s 129us/step - loss: 7.7456 - acc: 0.5194
Epoch 7/30
2545/2545 [==============================] - 0s 128us/step - loss: 7.7456 - acc: 0.5194
Epoch 8/30
2545/2545 [==============================] - 0s 130us/step - loss: 7.7456 - acc: 0.5194
Epoch 9/30
2545/2545 [==============================] - 0s 155us/step - loss: 7.7456 - acc: 0.5194
Epoch 10/30
2545/2545 [==============================] - 0s 136us/step - loss: 7.7456 - acc: 0.5194
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 218445.87it/s]

 32%|█████████████████████████▌                                                       | 63/200 [00:05<01:28,  1.55it/s]


100%|███████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 95565.07it/s]

100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 235245.08it/s]

100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 611706.94it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 510131.60it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 85059.20it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 170152.29it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 6715.26it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 255157.25it/s]

100%|███████████████████████████

Epoch 1/30
3050/3050 [==============================] - 0s 153us/step - loss: 7.8424 - acc: 0.5134
Epoch 2/30
3050/3050 [==============================] - 0s 138us/step - loss: 7.8424 - acc: 0.5134
Epoch 3/30
3050/3050 [==============================] - 0s 132us/step - loss: 7.8424 - acc: 0.5134
Epoch 4/30
3050/3050 [==============================] - 0s 142us/step - loss: 7.8424 - acc: 0.5134
Epoch 5/30
3050/3050 [==============================] - 0s 131us/step - loss: 7.8424 - acc: 0.5134
Epoch 6/30
3050/3050 [==============================] - 0s 147us/step - loss: 7.8424 - acc: 0.5134
Epoch 7/30
3050/3050 [==============================] - 0s 147us/step - loss: 7.8424 - acc: 0.5134
Epoch 8/30
3050/3050 [==============================] - 0s 141us/step - loss: 7.8424 - acc: 0.5134
Epoch 9/30
3050/3050 [==============================] - 0s 144us/step - loss: 7.8424 - acc: 0.5134
Epoch 10/30
3050/3050 [==============================] - 0s 137us/step - loss: 7.8424 - acc: 0.5134
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
for i in range(len(metrics_all)):
    print(metrics_all[i])

                      roc_auc     tn     fp     fn     tp       map  \
knn_clf              0.615693  173.0  101.0   94.0  141.0  0.534263   
random_forest        0.746746  218.0   56.0   71.0  164.0  0.659721   
logistic_regression  0.589882  195.0   79.0  125.0  110.0  0.518010   
svc                  0.525827  266.0    8.0  216.0   19.0  0.481257   
xgboost              0.721199  204.0   70.0   71.0  164.0  0.628596   
simple-NN            0.500000  274.0    0.0  235.0    0.0  0.461690   

                     precision    recall  accuracy  
knn_clf               0.582645  0.600000  0.616896  
random_forest         0.745455  0.697872  0.750491  
logistic_regression   0.582011  0.468085  0.599214  
svc                   0.703704  0.080851  0.559921  
xgboost               0.700855  0.697872  0.722986  
simple-NN             0.000000  0.000000  0.538310  
                      roc_auc     tn    fp     fn     tp       map  precision  \
knn_clf              0.659077  200.0  95.0   77.0 